In [1]:
#%matplotlib ipympl
import numpy as np
import pandas as pd
import os
import numba as nb
import random

from typing import Tuple, List

import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import normalize
import random

#Plotting
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from textwrap import wrap

#Maths
from scipy.integrate import solve_ivp
import scipy.spatial as sp

# Parallelisation
import contextlib 
import joblib
from joblib import Parallel, delayed
from tqdm.notebook import tqdm

mycol = ["red", "darkorange", "yellow", "limegreen", "dodgerblue", "darkviolet", "deeppink" ]


In [2]:
@contextlib.contextmanager
def tqdm_joblib(tqdm_object):
    """Context manager to patch joblib to report into tqdm progress bar given as argument"""
    class TqdmBatchCompletionCallback(joblib.parallel.BatchCompletionCallBack):
        def __call__(self, *args, **kwargs):
            tqdm_object.update(n=self.batch_size)
            return super().__call__(*args, **kwargs)

    old_batch_callback = joblib.parallel.BatchCompletionCallBack
    joblib.parallel.BatchCompletionCallBack = TqdmBatchCompletionCallback
    try:
        yield tqdm_object
    finally:
        joblib.parallel.BatchCompletionCallBack = old_batch_callback
        tqdm_object.close()

In [3]:
@nb.njit(nb.float64[:](nb.float64[:, :], nb.int32, nb.int32, nb.int32, nb.float64[:]))
def partials(positions: np.array, i: int, j: int, k: int, area: np.array) -> np.array:
    """
    Calculate the partial derivatives required when calculating the force due to preferred areas (BM contribution).
    Args:
        positions (np.array): A 2D numpy array of shape (N_cells, 3) representing the positions of cell centers.
            positions[i,:] = x,y,z positions of center of cell i.
        i (int): The index of the first vertex of the simplex.
        j (int): The index of the second vertex of the simplex.
        k (int): The index of the third vertex of the simplex.
        area (np.array): A numpy array containing the area of the simplex defined by vertices i, j, and k.
    Returns:
        np.array: A numpy array representing 1/(2*area) * partial derivative of the area with respect to xi, yi, and zi.
    """
    x_i = positions[i,0] 
    y_i = positions[i,1]
    z_i = positions[i,2]
    x_j = positions[j,0] 
    y_j = positions[j,1]
    z_j = positions[j,2]
    x_k = positions[k,0] 
    y_k = positions[k,1]
    z_k = positions[k,2] 

    partial_xi =(y_j - y_k)*((x_j-x_i)*(y_k-y_i) - (x_k-x_i)*(y_j-y_i) )+ (z_k-z_j)*(-(x_j-x_i )*(z_k-z_i) + (x_k-x_i)*(z_j-z_i))
    partial_yi =(-x_j + x_k)*((x_j-x_i)*(y_k-y_i) - (x_k-x_i)*(y_j-y_i))+ (z_j - z_k)*((y_j-y_i)*(z_k-z_i) - (y_k-y_i)*(z_j-z_i))
    partial_zi =(-x_j + x_k)*((x_j-x_i)*(z_k-z_i) - (x_k-x_i)*(z_j-z_i)) + (y_k-y_j)*((y_j-y_i)*(z_k-z_i) - (y_k-y_i)*(z_j-z_i))
    
    return 1/(2*area)* np.array([partial_xi, partial_yi, partial_zi])

@nb.njit(nb.float64[:, :](nb.int32[:, :], nb.int32))
def get_neighbours(simplices, N_bodies):
    """
    Determine the neighbors for each body using simplices calculated from a Delaunay triangulation.
    This function creates a matrix where each row corresponds to a body and each column contains the indices of its neighbors.
    The matrix is filled with NaNs initially, and valid neighbor indices are populated based on the simplices provided.

    Args:
        simplices (np.array): A 2D numpy array of simplices of shape (N_simplices, 4), calculated as simplicies = sp.Delaunay(positions).simplices.
            Indices of the points forming the simplices in the triangulation.
        N_bodies (int): The total number of bodies in the system.
            Before lumen formation, N_bodies = N_cells. After lumen formation, N_bodies = N_cells + 1 (lumen)

    Returns:
        neighbours (np.array): A 2D numpy array where each row represents a body and each entry in the row is an index of a neighbor.
            The array dimensions are (N_bodies, N_bodies).
            neighbours[i,:] = indices of cells neighbouring cell i
            Neighbours of each cell are filled left to right, so neighbours[;,j] = np.NaN indicates no further neighbours
    """
    neighbours = np.empty((N_bodies, N_bodies))
    neighbours[:] = np.nan
    for i in range(np.shape(simplices)[0]):
        simplex = simplices[i]
        for j in range(4):
            for k in range(4):
                if j != k and simplex[j] not in neighbours[simplex[k]]:
                    for m in range(N_bodies):
                        if np.isnan(neighbours[simplex[k], m]):
                            neighbours[simplex[k], m] = simplex[j]
                            break
    return neighbours


@nb.njit(nb.types.UniTuple(nb.float64[:, :], 2)(nb.float64[:, :], nb.float64[:, :], nb.float64[:, :], nb.float64[:, :], nb.float64, nb.float64[:, :], nb.int32, nb.int32[:, :], nb.float64[:, :], nb.float64[:, :], nb.float64[:, :], nb.float64[:, :], nb.int32, nb.float64, nb.float64))
def calculate_force(positions, neighbours, ages, lifetimes, alpha, morse_force_matrix, N_bodies, simplices, areas, equations, bm_pressure_force_matrix, bm_area_force_matrix, beta, P_star, A_eq_star):
    """
    Calculate the combined forces (Morse + BM_Area + BM_Pressure) acting on each body in the system.

    Args:
        positions (np.array): A 2D numpy array of shape (N_bodies, 3) containing the positions of each cell centre.
            positions[i,:] = x,y,z positions of center of cell i.
        neighbours (np.array): A 2D numpy array of shape (N_bodies, N_bodies) indicating the neighbours of each body.
            neighbours[i,:] = indices of cells neighbouring cell i.
            neighbours[;,j] = np.NaN indicates no further neighbours.
        ages (np.array): A 1D numpy array of shape (N_bodies, 1) containing the age of each body.
        lifetimes (np.array): A 1D numpy array of shape (N_bodies, 1) containing the lifetime of each body.
        N_bodies (int): The number of bodies in the system.
            Before lumen creation, N_bodies = N_cells. After lumen formation, N_bodies = N_cells + 1 (lumen).
        MORSE Args:
            alpha (float): A coefficient used in the calculation of Morse forces.
            morse_force_matrix (np.array): A 2D numpy array of shape (N_bodies, N_bodies) used to store calculated Morse forces.
        BM Args:
            simplices (np.array): A 3D array of ints, shape (N_simplices, 3)) Indices of points forming the simplical facets of the convex hull.
            areas (np.array): A 1D numpy array containing the area of each simplex.
            equations (np.array): Calculated from sp.ConvexHull(positions).equations
                ndarray of double, shape (nsimplex, ndim+2)
                [normal, offset] forming the hyperplane equation of the facet on the paraboloid
            bm_pressure_force_matrix (np.array): A 2D numpy array to store BM pressure forces.
            bm_area_force_matrix (np.array): A 2D numpy array to store BM area forces.
            beta (float): A coefficient used in BM force calculations.
            P_star (float): Preferred pressure parameter.
            A_eq_star (float): Equilibrium area parameter.

    Returns:
        force_matrix (np.array): A 2D numpy array of shape (N_bodies, 3) containing the calculated forces for each body.
    """
    for ii in range(N_bodies):
        for jj in neighbours[ii]:
            if np.isnan(jj) or ii <= jj:
                continue
            else:   
                jj = int(jj)
                r_ij_star = positions[jj,:] - positions[ii,:]
                r_mag = np.sqrt(np.dot(r_ij_star,r_ij_star))
                J = 2 + ((np.cbrt(2)-1)* ((ages[ii]/lifetimes[ii])+(ages[jj]/lifetimes[jj]))) ###################POSSIBLY SPEED THIS UP BY CALCULATING ALL JS AT ONCE IN MATRIX
                F_morse_mag = -2*(np.exp(alpha*(r_ij_star - J)) - np.exp(2*alpha*(r_ij_star-J)))
                F_morse = F_morse_mag *  (r_ij_star/r_mag)
                morse_force_matrix[ii,:] -= F_morse
                morse_force_matrix[jj,:] += F_morse

    # CALCULATE FORCE DUE TO PREFERRED SIMPLEX AREAS (BM CONTRIBUTION)
    for count, face in enumerate(simplices): # face has the form [i j k], where i j and k are the indices of the cells that define one simplex
        areas[count] = 0.5* np.linalg.norm(np.cross(positions[face[1]]-positions[face[0]],positions[face[2]]-positions[face[0]])) #area of each face
        for index in range(3):
            bm_pressure_force_matrix[face[(index%3)]] += (beta/3)* P_star * areas[count] * equations[count,0:3]
            bm_area_force_matrix[face[index]] -= beta * (areas[count]-A_eq_star) * partials(positions, face[index], face[(index+1)%3], face[(index+2)%3], areas[count])     

    force_matrix = bm_pressure_force_matrix + bm_area_force_matrix + morse_force_matrix

    return force_matrix, areas
    


In [4]:
def calculate_total_forces():
    """
    Calculate and update the total forces acting on the system, including Morse forces and forces due to preferred BM area and BM inner pressure.
    This function computes the forces for each body based, updates the global force matrix and tracks various properties for plotting and analysis.

    This function is split into subfunctions so that jit compilation can be used on the smaller tasks:
        1. get_neighbours(simplices, N_bodies)  @nb.njit(nb.float64[:, :](nb.int32[:, :], nb.int32))
        2. calculate_force(positions, neighbours, ages, lifetimes, alpha, morse_force_matrix, N_bodies, simplices, areas, equations, bm_pressure_force_matrix, bm_area_force_matrix, beta, P_star, A_eq_star)
            @nb.njit(nb.float64[:, :](nb.float64[:, :], nb.float64[:, :], nb.float64[:, :], nb.float64[:, :], nb.float64, nb.float64[:, :], nb.int32, nb.int32[:, :], nb.float64[:, :], nb.float64[:, :], nb.float64[:, :], nb.float64[:, :], nb.int32, nb.float64, nb.float64))
            2b. partials(positions: np.array, i: int, j: int, k: int, area: np.array)  @nb.njit(nb.float64[:](nb.float64[:, :], nb.int32, nb.int32, nb.int32, nb.float64[:]))
    
    Global Variables:
        neighbours (np.array): A 2D numpy array of shape (N_bodies, N_bodies) indicating the neighbours of each body.
            neighbours[i,:] = indices of cells neighbouring cell i.
            neighbours[;,j] = np.NaN indicates no further neighbours.
        current_area: array of current areas of convex hull
        force_matrix (np.array): A matrix of shape (N_bodies, 3) to store the calculated forces for each body.
        hull (sp.ConvexHull): The convex hull of the positions, used for force calculations.
        bm_area_force_matrix, bm_pressure_force_matrix, morse_force_matrix: Matrices each of shape (N_bodies, 3) used to store different force contributions.
        all_forces, average_areas, all_volumes, all_lumen_volumes, all_preferred_areas, all_positions, all_N_bodies: Lists each of length (N_timesteps), used to track simulation properties at each timestep.
        A_eq_star, inner_pressure (float): Parameters used in force calculations.

    Modifies:
        Updates the global force_matrix and other global lists used for tracking simulation properties.

    Note:
        This function relies on other functions like 'calculate_force' and 'get_neighbours' to perform its calculations.
    """
    global neighbours, force_matrix, hull
    global current_areas
    global all_forces, average_areas, all_volumes, all_lumen_volumes, all_preferred_areas, all_positions
    global bm_area_force_matrix, bm_pressure_force_matrix, morse_force_matrix, all_N_bodies, A_eq_star, inner_pressure

    hull = sp.ConvexHull(positions)
###############################################################################################################
## CALCULATE THE NEIGHBOURS
    dela = sp.Delaunay(positions)
    neighbours = np.empty((N_bodies, N_bodies))
    neighbours[:] = np.nan

    neighbours = get_neighbours(dela.simplices, N_bodies)

###############################################################################################################

###############################################################################################################
# CALCULATE THE FORCES
# CREATE A BLANK FORCE MATRIX
    force_matrix = np.zeros((N_bodies, 3)) #N_bodies = N_cells + N_lumen(=0/1)
    bm_pressure_force_matrix = np.zeros((N_bodies, 3))
    bm_area_force_matrix = np.zeros((N_bodies,3))
    morse_force_matrix = np.zeros((N_bodies, 3))
# CALCULTE MORSE FORCE + FORCE DUE TO INNER PRESSURE
    
    areas = np.zeros([len(hull.simplices),1])

    force_matrix, areas = calculate_force(positions, neighbours, ages, lifetimes, alpha, morse_force_matrix, N_bodies, hull.simplices, areas, hull.equations, bm_pressure_force_matrix, bm_area_force_matrix, beta, P_star, A_eq_star)

    # TRACK PROPERTIES FOR PLOTTING (OPTIONAL)
    all_positions.append(positions)
    all_forces.append(force_matrix)
    all_volumes.append(hull.volume)
    current_areas = areas
    average_areas.append(areas.mean())
    all_lumen_volumes.append(lumen_volume)
    all_preferred_areas.append(A_eq_star)


In [5]:
def r_dot(t, r):
    """
    Function to
    Parameters:
        t: tuple timespan, (t_min, t_max), over which the solver should run
        r: 1d list of the positions at a timepoint, array-like initial state    
    Returns:
        drdt: array-like, total forces experienced by each body, converted to a list (=fun in scipy documentation)
    """
    global neighbours, force_matrix, positions, ages, all_ages, A_eq_star
    global all_t_values, all_energies, all_lumen_volumes, all_forces, all_areas, all_volumes

    # r is a 1d list of the positions at a timepoint
    positions = np.asarray(r).reshape(N_bodies,3)
    time_increment = all_t_values[-1]-all_t_values[-2]

    calculate_total_forces()
    current_ages = ages   
    all_t_values.append(t)
    A_eq_star = (hull.area / (hull.simplices.shape[0])) * A_eq_star_scaling
    all_preferred_areas.append(A_eq_star)
    if lumen_status == False:
        ages = current_ages + time_increment
    else:
        ages[:-1] = current_ages[:-1] + time_increment
        ages[-1] = current_ages[-1]  
           
    all_ages.append(ages)
    round_percent = int(round((all_t_values[-1]/t_max)*50))
    print(round_percent*"|"+ (50-round_percent)*"-" + "t={}".format(round(all_t_values[-1],2)), end = "\r")
    ##########################################################
    drdt = force_matrix.flatten().tolist()
    return drdt

In [6]:
def cell_joins_lumen():
    global ages, positions, hull, lumen_status, N_bodies, last_event_time, all_lumen_volumes, lumen_radius, lifetimes, lumen_volume, fail_cause
    print("CELL JOINS LUMEN TRIGGERED AT TIME {} WHEN THERE ARE {} BODIES TOTAL AND LUMEN STATUS IS {}".format(all_t_values[-1], N_bodies, lumen_status))
    ####the lumen will still have an age (to define its volume)
    new_lumen_volume = 0
    numerator, denominator = 0,0

    # After the extra cells have joined the lumen, we need to have at least 2 cells remaining in order to position the lumen
    # and define a ConvexHull
    if lumen_status == False:
        new_exterior_cells = hull.vertices
    if lumen_status == True:
        existing_lumen_radius = r_min*(1 + ((2**(1/3))-1)  * ((ages[-1,0])/mean_lifetime))
        print("existing lumen radius = {}".format(existing_lumen_radius))
        print("radius scaling = {}".format(radius_scaling))
        # Calculate the distance between all bodies and the lumen (position[-1])
        distances = np.linalg.norm(positions - positions[-1], axis=1) 
        print("distances are {}".format(distances))
        bodies_in_centre = [i for i in range(N_bodies-1) if distances[i] <= (radius_scaling * existing_lumen_radius)] # this includes the current lumen
        bodies_in_centre.append(N_bodies-1) #so bodies in centre = existing lumen + inner cells
        new_exterior_cells = [i for i in range(N_bodies) if i not in bodies_in_centre]
    new_cell_number = len(new_exterior_cells)
    
    if new_cell_number >=2:
        # IF THIS IS THE CREATION OF THE LUMEN
        if lumen_status == False:
            print("CURRENTLY NO LUMEN")
            N_cells = len(hull.vertices) 
            new_positions = np.zeros((N_cells + 1, 3))
            new_ages = np.zeros((N_cells + 1, 1))
            new_lifetimes = np.zeros((N_cells + 1, 1))
            bodies_in_centre = [i for i in range(N_bodies) if i not in hull.vertices]
            print("THERE ARE {} BODIES IN THE CENTRE".format(len(bodies_in_centre)))
            print("CURRENT POSITIONS {}".format(positions))
            """
            Lumen creation is triggered by cells not in the convex hull, depending on the initial configuration, this could be >1 cell
            First cell contributes all of its volume to the lumen, and subsequent cells contribute (volume_scaling * cell_volume)
            """
            for i, body in enumerate(bodies_in_centre):
                mass = r_min * (4/3) * np.pi * (1+ ((np.cbrt(2)-1)*ages[body,0]/lifetimes[body,0]))**3
                if i== 0:
                    print("First cell in the new lumen is added, i = {}, mass = {}, positions ={}".format(i, mass, positions[body]))
                    new_lumen_volume += mass
                else:
                    print("Extra cell added to new lumen, i = {}, mass = {}, positions ={}, volume scale = {}".format(i, mass, positions[body], volume_scaling))
                    new_lumen_volume += volume_scaling * mass

        ###############################################################################################################

        ###############################################################################################################
        # IF THERE IS AN EXISTING LUMEN THAT WE ARE ADDING TO
        else:
            print("CURRENTLY THERE IS A LUMEN")
            print("CURRENT POSITIONS {}".format(positions))
            print("THERE IS ALREADY A LUMEN AT POSITION {}. THE BODIES IN CENTRE ARE {}".format(N_bodies-1, bodies_in_centre))
            print("THERE ARE {} BODIES IN THE CENTRE".format(len(bodies_in_centre)))

            new_N_bodies = new_cell_number + 1 # remaining cells + lumen
            new_positions = np.zeros((new_N_bodies, 3))
            new_ages = np.zeros((new_N_bodies, 1))
            new_lifetimes = np.zeros((new_N_bodies, 1))
            print("remaining cells are {} and inner cells (+lumen) are {}".format(new_exterior_cells, bodies_in_centre))      
            ################## Calculate the new lumen volume##################
            # Existing lumen contributes all of its lumen, and additional cells contribute (volume_scaling * cell_volume)
            #TO DO -> Change mass to volume for clarity
            for i, body in enumerate(bodies_in_centre):
                if body != N_bodies-1:
                    mass = r_min * (4/3) * np.pi * (1+ ((np.cbrt(2)-1)*ages[body,0]/lifetimes[body,0]))**3
                    added_mass = mass * volume_scaling
                    new_lumen_volume += added_mass
                    print("Adding a new cell, i,body = {},{} with age {} position {} and volume {} x scaling {}".format(i, body, ages[body,0], positions[body], mass, volume_scaling ))
                else:
                    mass = r_min * (4/3) * np.pi * (1+ ((np.cbrt(2)-1)*ages[-1,0]/mean_lifetime))**3
                    added_mass = mass
                    new_lumen_volume += added_mass #add the existing lumen volume
                    print("Adding the existing lumen, i,body = {},{} with age {} position {} and volume {} ".format(i, body, ages[body,0], positions[body], mass ))
 
        
        print("NOW CALCULATING THE POSITION OF THE NEW LUMEN")              
        for i, body in enumerate(new_exterior_cells):  
            mass = r_min * (4/3) * np.pi * (1+ ((np.cbrt(2)-1)*ages[body,0]/lifetimes[body]))**3
            numerator += mass * positions[body]
            denominator += mass
            print("Adding new exterior cell i={}, position = {}, mass ={}".format(body, positions[body], mass))

        ##################Reorder the Positions and Ages##################
        for count, cell_index in enumerate(new_exterior_cells):
            new_positions[count] = positions[cell_index]
            new_ages[count] = ages[cell_index]
            new_lifetimes[count] = lifetimes[cell_index]

        lumen_scale_age = (mean_lifetime/(np.cbrt(2) -1)) *(((1/r_min) * np.cbrt(3*new_lumen_volume * 0.25 * (1/np.pi)))-1)
        lumen_radius = (0.75 * new_lumen_volume * (1/np.pi))**(1/3)
        lumen_volume = new_lumen_volume
        
        new_ages[-1] = lumen_scale_age 
        new_lifetimes[-1] = mean_lifetime
        new_positions[-1,:] = numerator/denominator 
        print("NEW POSITION OF LUMEN + {}".format(new_positions)) 
        ages = new_ages
        lifetimes =new_lifetimes
        N_bodies = ages.shape[0]
        positions = new_positions
        lumen_status = True
    
        try:
            print("DONE LUMEN JOINING, TRY JOIN LUMEN")
            hull=sp.ConvexHull(positions)
            print("SUCCESSFUL HULL ATTEMPT")
            last_event_time = all_t_values[-1]
            all_lumen_volumes.append(lumen_volume)

            print("THE LUMEN NOW HAS  AGE{} and volume {}".format(lumen_scale_age, lumen_volume))    
            print("AFTER LUMEN JOINING THERE ARE {} BODIES AND THE LAST EVENT TIME IS {}".format(N_bodies, last_event_time))
            print("AFTER LUMEN JOINING THE AGES ARE", ages)
            print("AFTER LUMEN JOINING THE LIFETIMES ARE", lifetimes)
        except:
            print("HULL FAILED, positions = {}, ages ={}".format(positions, ages))
            fail_cause = "all_lumen"
            

    else:
        print("TOO FEW CELLS TO ATTEMPT LUMEN FORMATION")
        fail_cause = "all_lumen"

In [7]:
def perform_divisions(): 
    global ages, positions, N_bodies, hull, last_event_time, lifetimes
    print("PERFORM DIVISIONS TRIGGERED AT TIME {} WHEN THERE ARE {} BODIES INITIALLY".format(all_t_values[-1], N_bodies))
    # print("WHEN DIVISION IS TRIGGERED THE AGES AND LIFETIMES ARE {}".format([(ages[i,0], lifetimes[i,0]) for i in range(N_bodies)]))
    N_old = N_bodies
    old_ages = ages
    old_positions = positions
    ################## Find out how many cells need to divide ##################
    # If there is no lumen, any age> lifetime denotes an old cell
    if lumen_status == False:
        number_dividing = ages[ages>lifetimes].size
        enumerator = ages
        start_index = 0
    # If there is a lumen, its age can exceed the lifetime
    else:
        number_dividing = ages[:-1][ages[:-1]>lifetimes[:-1]].size
        enumerator = ages[:-1]
        start_index = -1
    # print("THERE ARE {} CELLS DIVIDING".format(number_dividing))
    
    
    N_bodies =  N_old + number_dividing
    new_positions = np.zeros((N_bodies,3))
    new_ages = np.zeros((N_bodies, 1))
    new_lifetimes = np.zeros((N_bodies, 1))
    j = 0 #keep track of the number of dividing cells we've dealt with 
    for i, age in enumerate(enumerator): ##############CHECKKKKKKKKKKKKKKK
        if age < lifetimes[i]:
            new_positions[i] = positions[i]
            new_ages[i] = ages[i]
            new_lifetimes[i] = lifetimes[i]
        else:
            new_ages[i] = 0
            new_lifetimes[i] = np.random.normal(mean_lifetime, lifetime_std)
            new_ages[N_old + j+ start_index] = 0
            new_lifetimes[N_old + j + start_index] = np.random.normal(mean_lifetime, lifetime_std)
            #print("N_old, j =", N_old, j, "new ages [N_old + j]", new_ages[N_old + j], "new_lifetimes[N_old + j]",new_lifetimes[N_old + j] )
            theta_rand = np.random.uniform(0,360)
            phi_rand = np.random.uniform(0,360)

            # print("CURRENT NEW LIFETIMES ARE", new_lifetimes)
            new_positions[i][0] = positions[i][0] + r_min*np.cos(phi_rand)* np.sin(theta_rand)
            new_positions[i][1] = positions[i][1] +  r_min * np.sin(phi_rand) * np.sin(theta_rand)
            new_positions[i][2] = positions[i][2] +  r_min * np.cos(theta_rand)
            new_positions[N_old + j + start_index][0] =  new_positions[i][0]-(2*r_min*np.cos(phi_rand)* np.sin(theta_rand))
            new_positions[N_old + j + start_index][1] = new_positions[i][1]-(2*r_min * np.sin(phi_rand) * np.sin(theta_rand))
            new_positions[N_old + j+ start_index][2] = new_positions[i][2] - (2*r_min * np.cos(theta_rand))        
            j += 1


    if lumen_status == True:
        # If there's a lumen, its age and position should stay at position [-1]
        new_ages[-1] = old_ages[-1]
        new_lifetimes[-1] = mean_lifetime
        new_positions[-1] = old_positions[-1]
    positions = new_positions
    ages = new_ages
    lifetimes = new_lifetimes
    hull = sp.ConvexHull(positions)
    N_bodies = ages.shape[0]
    last_event_time = all_t_values[-1]
    print("THE AGES ARE NOW", ages)
    # print("AFTER DIVISION THERE ARE {} BODIES AND THE LAST EVENT TIME IS {}".format(N_bodies, last_event_time))
    # print("AFTER DIVISION THE AGES ARE", ages)
    # print("AFTER DIVISION THE LIFETIMES ARE", lifetimes)
    # print("LUMEN STATUS =", lumen_status)

In [8]:
def events(t,r): #event triggered if outcome == 0 (code breaks)
    # If there hasn't been much time since the last event, keep going without performing an action
    # Unless the last event time was 0 (i.e. there has been no event)
    if last_event_time != 0 and (all_t_values[-1] - last_event_time) <= mean_lifetime/1000 :
        return 1
    ################## TOO OLD ##################
    # Check if the age of any cell exceeds the lifetime
    # The "age" of the lumen is just used to calculate its radius, so its "age" can exceed the lifetime
    if lumen_status == False:
        too_old = int(not any (ages>=lifetimes))
    else:
        too_old = int(not any (ages[0:-1] > lifetimes[0:-1]))
    if too_old == 0:
        return 0
    ##################SIMPLICES BELOW THE CUTOFF VALUE##################
    if any (current_areas) <  A_eq_star:
        print("AREAS TOO SMALL")
        return 0
    #######LUMEN IS ALL
    if positions.shape[0]<=3:
        print("LUMEN IS ALL")
        return 0
    ################## JOIN LUMEN ##################
    # If there's no lumen, any cell that falls inside the convex hull becomes the lumen
    # If there's already a lumen, cells join when their centre likes within (radius scaling * lumen radius) of the lumen centre
    if lumen_status == False:
        inside_hull = int(N_bodies == len(hull.vertices))
    else:
        distances = np.linalg.norm(positions - positions[-1], axis=1) 
        cells_inside = [i for i in range(N_bodies) if distances[i] <= radius_scaling * lumen_radius]
        if len(cells_inside) > 1:
            inside_hull = 0
        else:
            inside_hull = 1
    return inside_hull

events.terminal = True

In [9]:
def run_simulation_and_save(run_number, params, alter="All"):  
    print("##### RUN NUMBER {} BEGINS #####".format(run_number)) 

###########################
##Force Parameters
    global r_min, beta, alpha, P_star, mean_lifetime, ages, A_eq_star_scaling, A_eq_star
    r_min = params[0]
    beta = params[1]
    alpha = params[2] * r_min
    A_eq_star_scaling = params[3] ###############this is now the scaling! Like the area = (hull.area / N_simplices) * scale
    P_star = params[4]
    mean_lifetime = params[5]
    A_eq_star = 0.1

# values to do with the simulation that don't change
    global delta_t_max, N_bodies, t_max, boxsize
    delta_t_max = 0.001
    N_bodies = 4
    boxsize = 5
    t_max = mean_lifetime * 4
    print(t_max)

    global lifetime_std, lifetimes
    lifetime_std = mean_lifetime/100
    lifetimes = np.random.normal(mean_lifetime, lifetime_std, size=(N_bodies, 1))
    ages = lifetimes * np.random.rand(N_bodies, 1)

# Parameters that control when a cell joins the lumen
    global lumen_volume
    lumen_volume = 0
    global volume_scaling, radius_scaling, lumen_status, all_lumen_volumes
    volume_scaling = params[6]# When a cell joins an existing lumen, the lumen volume will increase by (volume_scaling * volume_of_joining_cell)
    radius_scaling = params[7] # A cell will join the lumen when its centre lies within (radius_scaling * lumen_radius) of the lumen centre
    lumen_status = False # Assume there's no lumen, this is updated when a lumen forms
    all_lumen_volumes = []

# values that update as we progress - these are just a snapshot of the value at the current time
    global positions, force_matrix, initial_positions, hull
    positions = boxsize * np.random.rand(N_bodies,3) * random.choice([1,-1])
    positions[0] = ([1/2,-np.sqrt(3)/6,0])
    positions[1] = ([-1/2,-np.sqrt(3)/6,0])
    positions[2] = ([0,np.sqrt(3)/3,0])
    positions[3] = ([0,0,np.sqrt(3/6)])
    force_matrix = np.zeros((N_bodies, 3))
    hull = sp.ConvexHull(positions)

# values that we track - these are lists that we append to at each time point
    global  all_forces, average_areas, all_volumes, all_positions, all_ages, all_preferred_areas
    all_forces, average_areas,  all_volumes, all_positions, all_ages, all_preferred_areas = [], [], [], [], [], []

    global current_areas
    current_areas = np.inf

    global all_t_values, last_event_time
    all_t_values = [0,0]
    last_event_time = 0

    global fail_cause
    fail_cause = 0

    t_min = 0
    while t_min < t_max:
        try:
            sol = solve_ivp(r_dot, (t_min, t_max), positions.flatten(), method='RK45', first_step=0.0003, max_step=delta_t_max, events=(events))
            t_min = max(sol.t)
            if t_min > t_max:
                fail_cause = "success"
                break
            if any (current_areas) <  A_eq_star:
                fail_cause = "small_area"
                print("BROKE DUE TO SMALL AREAS")
                break
            ###################### CURRENTLY NO LUMEN ######################
            if lumen_status == False:
                if len(hull.vertices) != N_bodies:
                    print("THERE ARE {} BODIES AND THE VERTICES ARE {}".format(N_bodies, hull.vertices))
                    cell_joins_lumen()
                    if fail_cause == "all_lumen":
                        break
                elif any(ages >= lifetimes):   
                    perform_divisions()
            ###################### THERE IS AN EXISTING LUMEN ######################
            else:
                if len([i for i in range(N_bodies) if np.linalg.norm(positions - positions[-1], axis=1) [i] <= (lumen_radius*radius_scaling)]) > 1: # if any of the bodies has a distance that's too close to the lumen
                    cell_joins_lumen()
                    if fail_cause == "all_lumen":
                        break

                elif any (ages[0:-1] > lifetimes[0:-1]):
                    perform_divisions()
                else:
                    fail_cause = "unknown"
                    print("ISSUE HERE")
        except:
            print("fail")
            print("posiions = ", positions)
            break
    del all_t_values[0:2]

    global final_positions
    final_positions = positions

 
    df = pd.DataFrame(list(zip(*[all_t_values, all_volumes, average_areas,  all_lumen_volumes, all_preferred_areas])))
    df.columns = ["t", "Cluster_Vol", "Areas", "Lumen_Volume", "Preferred_area"]
    df["Run_No"] = run_number

    df["r_min"] = r_min
    df["beta"] = beta
    df["alpha"] = alpha
    df["A_eq_star_scaling"] = A_eq_star_scaling
    df["P_star"] = P_star
    df["mean_lifetime"] = mean_lifetime
    df["lumen_volume_scaling"] = volume_scaling
    df["lumen_radius_scaling"] = radius_scaling
    df["end_reason"] = fail_cause
    
    df.to_parquet(".\\Animating_Output\\BM_based_on_area\\Alter{}_Run{}.parquet".format(alter, run_number))
    np.save('.\\Animating_Output\\BM_based_on_area\\Alter{}_positions_{}.npy'.format(alter, run_number), np.array(all_positions, dtype=object), allow_pickle=True)
    np.save('.\\Animating_Output\\BM_based_on_area\\Alter{}_ages_{}.npy'.format(alter, run_number), np.array(all_ages, dtype=object), allow_pickle=True)
    return df


In [10]:
import pickle
beta_run_range = np.linspace(0.001,10,10)
alpha_over_min_run_range = np.linspace(0.001,10,10)
#A_eq_run_range = np.linspace(0.01, 0.95, 10) full space
A_eq_run_range = np.linspace(0.01, 0.55, 10)
P_star_run_range = np.linspace(0.1, 10, 10)
volume_scaling_run_range = np.linspace(0.01, 0.5, 8)
#radius_scaling_run_range = np.linspace(0.6, 1.6, 10) full space
radius_scaling_run_range = np.linspace(0.6, 1.2, 10)
#r_min is always 1 and lifetime is always 5
parameters = [(1, b, a, A, p, 5, V, r) for b in beta_run_range for a in alpha_over_min_run_range for A in A_eq_run_range for p in P_star_run_range for V in volume_scaling_run_range for r in radius_scaling_run_range]
random.shuffle(parameters)
len(parameters)

800000

In [11]:
with tqdm_joblib(tqdm(desc="My Simulation", total=len(parameters))) as progress_bar:
    Parallel(n_jobs=3)(delayed(run_simulation_and_save)(j+7575, params, "ALL") for j, params in enumerate(parameters))

My Simulation:   0%|          | 0/800000 [00:00<?, ?it/s]